<a href="https://colab.research.google.com/github/ubermen/anomaly_detector/blob/master/nlog_quality_anom_detection_alpha.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!rm -rf anomaly_detector
!git clone https://github.com/ubermen/anomaly_detector.git
%cd anomaly_detector
!ls
from preprocessor import Preprocessor
from testor import TestUtil
from vae_on_cnn import VariationalAutoencoder

import tensorflow as tf
tfd = tf.contrib.distributions
from google.colab import auth
auth.authenticate_user()

Cloning into 'anomaly_detector'...
remote: Enumerating objects: 89, done.
remote: Counting objects: 100% (89/89), done.
remote: Compressing objects: 100% (83/83), done.
remote: Total 89 (delta 53), reused 13 (delta 5), pack-reused 0
Unpacking objects: 100% (89/89), done.
/content/anomaly_detector
main.py  preprocessor.py  testor.py  vae_on_cnn.py


In [2]:
# data preprocessing
sequence_length = 20
encoding_size = 128
code_size = 2
  
preprocess = Preprocessor(sequence_length, encoding_size)
test_data = preprocess.extract_from_bigquery('test', 'bi-service-155107', 'bigpi_test', 'log_anomaly_globalsignin_devicemodel_20181004_test_shuffled', print_result=True)

test 15000


In [11]:
# Define the model 
vae = VariationalAutoencoder(sequence_length, encoding_size, code_size, kernel=(sequence_length,10))
data = tf.placeholder(tf.float32, [None, sequence_length, encoding_size])
prior = vae.make_prior()
posterior = vae.make_encoder(data)
code = posterior.sample()

# Define the loss.
decoder = vae.make_decoder(code)
likelihood = decoder.log_prob(data)
divergence = tfd.kl_divergence(posterior, prior)
elbo = tf.reduce_mean(likelihood - divergence)
optimize = tf.train.AdamOptimizer(learning_rate=0.001).minimize(-elbo)
anomaly_score = -likelihood

# checkpoint
checkpoint_dir = './training_checkpoints'
saver = tf.train.Saver()

test_util = TestUtil(anomaly_score, data)

latent_size 2
kernel (20, 10)
stride (1, 2)
final_conv [20, 32, 32]
input Tensor("78437f11-5e8a-42a0-8ed7-1a3be8d20b30/Reshape:0", shape=(?, 20, 128, 1), dtype=float32)
output Tensor("78437f11-5e8a-42a0-8ed7-1a3be8d20b30/conv2d/Relu:0", shape=(?, 20, 64, 16), dtype=float32)
output Tensor("78437f11-5e8a-42a0-8ed7-1a3be8d20b30/conv2d_1/Relu:0", shape=(?, 20, 32, 32), dtype=float32)
d_input Tensor("8ceb2632-7d68-4cdc-8fa0-ab117c34ed6e/Reshape:0", shape=(?, 20, 32, 32), dtype=float32)
d_output Tensor("8ceb2632-7d68-4cdc-8fa0-ab117c34ed6e/conv2d_transpose/BiasAdd:0", shape=(?, 20, 64, 16), dtype=float32)
d_output Tensor("8ceb2632-7d68-4cdc-8fa0-ab117c34ed6e/conv2d_transpose_1/BiasAdd:0", shape=(?, 20, 128, 1), dtype=float32)
d_output Tensor("8ceb2632-7d68-4cdc-8fa0-ab117c34ed6e/Reshape_1:0", shape=(?, 20, 128), dtype=float32)


In [0]:
# initialize
with tf.Session() as sess:
  sess.run(tf.global_variables_initializer())
  save_path = saver.save(sess, checkpoint_dir)

In [0]:
# train
total_epochs = 1
extract_batch_size = 100000
learning_batch_size = 1000

with tf.Session() as sess:
  is_training = True
  abnormal_samples = None
  saver.restore(sess, checkpoint_dir)
  for epoch in range(total_epochs):
    s = (epoch*100)%len(test_data)
    e = s + 100
    test_util.test('e_{:04d}'.format(epoch), sess, test_data[s:e])
    
    i = 0
    while True :
      e_start = i
      e_end = e_start + extract_batch_size - 1
      i = i + extract_batch_size
      train_batch_data = preprocess.extract_batch_from_bigquery('extract_batch', 'bi-service-155107', 'bigpi_test', 'log_anomaly_globalsignin_devicemodel_20181004_real', e_start, e_end)
      data_count = train_batch_data.shape[0]
      if data_count == 0 : break
      j = 0
      while j < data_count :
        l_start = j
        l_end = l_start + learning_batch_size - 1
        j = j + learning_batch_size
        feed = {data: train_batch_data[l_start:l_end]}
        sess.run(optimize, feed)
    print('finish epoch',epoch)
    save_path = saver.save(sess, checkpoint_dir)

INFO:tensorflow:Restoring parameters from ./training_checkpoints
[('LGE LG-F300L', 1774.0217), ('TCL 5017O', 1774.1135), ('Samsung SM-G610M', 1774.1473), ("K|C'\x19)cLu /\x1fvhVsP1i1", 1774.1575), (">]\n;\x0c'8\x1c\x04MRvoDuqP8~\x0f", 1774.1605), ('WpLiRuw\x13T07\x1f}\t\x02\x117XD', 1774.1609), ('E{C&E2\x0fPuL<\x1265\x0c3\x0cOX\x1e', 1774.1687), ('ynkuofejopphzcvktvkm', 1774.1702), ('/KOL\nI\x18z6n*/\x10eAMW_#&', 1774.1746), ('Motorola MotoG3-TE', 1774.1876), ('HUAWEI LND-AL30', 1774.1927), (":f}\\\x0b<'<$\x1eo\x1b&F\x1c,p&I", 1774.195), ('\x05\x08Tob\x12Ie8bH}l/\x0b> W|W', 1774.1982), ('ecajhamygbswypsgwffj', 1774.2097), ('#\x03\x18r4\x1dR\x1e7-H6\tt`\x13\x1f\x1e#6', 1774.2114), ('HUAWEI VNS-L23', 1774.2115), ('irsyeusnloaolxcujbqh', 1774.2134), ('wkvnylxsxhwbarepxish', 1774.214), ('Samsung SHV-E210L', 1774.218), ('DTAC dtacPhoneX3', 1774.219), ('npY\x07\x16j\x13\x04sFkbyh\x13YMm\x1dI', 1774.2211), ('Samsung SM-G925S', 1774.2277), ('rjfetcczgcqidgrpdfgf', 1774.2317), ('iyyfnrdeilwztic

In [15]:
#valid_codes = {'a1','ad','ae','af','ag','ai','al','am','ao','ar','as','at','au','aw','ax','az','ba','bb','bd','be','bf','bg','bh','bi','bj','bl','bm','bn','bo','bq','br','bs','bt','bw','by','bz','ca','cd','cf','cg','ch','ci','ck','cl','cm','cn','co','cr','cu','cv','cw','cx','cy','cz','de','dj','dk','dm','do','dz','ec','ee','eg','es','et','fi','fj','fm','fo','fr','ga','gb','gd','ge','gf','gg','gh','gi','gl','gm','gn','gp','gq','gr','gt','gu','gy','hk','hn','hr','ht','hu','id','ie','il','im','in','iq','ir','is','it','je','jm','jo','jp','ke','kg','kh','ki','km','kn','kr','kw','ky','kz','la','lb','lc','li','lk','lr','ls','lt','lu','lv','ly','ma','mc','md','me','mf','mg','mh','mk','ml','mm','mn','mo','mp','mq','mr','ms','mt','mu','mv','mw','mx','my','mz','na','nc','ne','ng','ni','nl','no','np','nu','nz','om','pa','pe','pf','pg','ph','pk','pl','pm','pr','ps','pt','pw','py','qa','re','ro','rs','ru','rw','sa','sb','sc','sd','se','sg','si','sk','sl','sm','sn','so','sr','ss','sv','sx','sy','sz','tc','tg','th','tj','tl','tm','tn','tr','tt','tw','tz','ua','ug','us','uy','uz','vc','ve','vg','vi','vn','vu','ws','xk','ye','yt','za','zm','zw'}
#valid_codes = {'_<?','_EG','_US','ar','ca','cs','da','de','en','es','fr','fa','fil','fi','he','hu','it','id','in','ja','ko','km','lo','ms','mi','my','nb','nl','pl','ps','pt','ro','ru','sv','th','tr','und','vi','zh','(n_(N','fil-PH','pam-PH','wuu-Hans-AM','yue-Hant-TW','yue-Hant-HK','yue-Hans-CN','yue-Hant-CN','ko_14','es-419','es_419','vi_:S','ar_AE','az_AZ','az_AZ_#Latn','ar-AE','ar_AR','ar_AZ','ca_AD','de_AT','de-AT','en_AS','en-AU','en-AD','en-AR','en-AM','en-AS','en_AI','en_AM','en_AU','es_AR','en-AE','en-AX','en-AF','en-AO','es-AR','en-AT','en_AE','en_AT','en_AR','en-AI','en-AW','en_AG','en-AL','en_AD','en-AZ','fa_AU','fa_AF','hy_AM','in_AU','ja-AE','ja-AR','ja-AL','ja-AQ','ja-AU','ja-AZ','ja-AF','ja-AS','ja-AT','ja-AC','ja-AM','ja-AW','ja-AO','ja-AI','ko-AU','ko_AU','ko-AT','ko-AL','ko-AQ','ko-AM','ko_AR','ko-AO','ko-AE','ko-AI','ko-AR','ko_AE','ko-AZ','pt_AO','ru-AU','ru-AM','ru-AE','ru_AM','ru_AZ','sv_AX','sq_AL','tr-AZ','th-AL','th-AU','th_AU','th-AT','vi-AD','vi-AG','vi-AS','vi-AQ','vi-AE','vi-AM','vi_AU','vi-AU','vi-AF','vi-AL','vi-AO','vi-AX','vi_AO','vi-AR','zh_AU','zh_AX','ar_BH','bs_BA','bg-BG','bo-BT','bg_BG','bs_BA_#Latn','bn_BD','de_BE','en-BG','en_BE','en_BG','en_BD','en_BH','es_BR','en_BS','en_BB','en-BM','en-BO','en-BN','en_BY','es_BO','en-BR','en_BR','en_BN','en-BB','en-BE','en_BZ','en_BI','en-BJ','en_BW','en_BA','es-BO','en-BD','en-BS','fr_BE','fr-BE','fr_BF','hr_BA','hr-BA','id-BE','ja-BH','ja-BL','ja-BE','ja_BR','ja-BR','ko_BR','ko-BO','ko-BQ','ko-BR','ko-BN','ko_BN','ko-BH','ko-BW','nl_BQ','nl_BE','nl-BE','pt_BR','pt-BR','pt-BO','ru-BY','ru-BA','ru_BY','sk-BE','sr_BA_#Latn','sr_BA','tr-BG','th-BF','vi-BZ','vi-BT','vi-BW','vi-BY','vi-BM','vi_BN','zh_BR','ar-CN','bo_CN','bo-CN','cs_CZ','cs-CZ','de_CH','de-CH','de-CN','en-CN','en_CY','en-CF','es-CO','en-CH','en_CZ','es-CL','en_CR','en_CL','en_CM','en-CL','es-CR','en-CZ','en_CA','es_CL','es_CO','en-CA','es_CR','en_CN','el_CY','es_CU','en_CX','en_CO','en_CH','es-CN','en_CC','es_CH','en-CW','en-CM','fr_CA','fr-CA','fr_CI','fr_CG','fr_CM','fr_CH','fr-CN','fr-CH','fr_CD','it_CH','it-CH','ja-CA','ja-CH','ja-CN','ko_CN','ko_CA','ko_CL','ko-CM','ko-CA','ko_CZ','ko-CN','ko-CH','ko-CL','ko-CZ','pt_CV','ru-CZ','ru-CN','ru_CN','ru-CA','rm_CH','tr_CY','tr-CA','th-CN','th_CN','th_CA','ug-CN','vi-CA','vi_CZ','vi-CO','vi-CN','vi-CZ','vi_CN','vi_CA','zh_CN','zh_CN_#Hans','zh_CF_#Hans','zh_CA','zh_CN_#Hant','ar_DZ','ar-DE','ar-DZ','de_DE','da-DK','da-DE','de-DE','da_DK','en_DK','en-DK','en_DZ','en_DO','en_DE','es_DO','en-DE','es-DO','en-DZ','es_DE','es-DE','fr-DZ','fr_DZ','fr-DE','hu-DE','it-DE','ja-DZ','ja-DE','ko-DE','ko-DK','ko-DZ','ko-DO','ko-DG','ru_DE','ru-DE','th-DE','tr-DE','tr_DE','vi-DE','vi-DK','vi-DZ','zh_DE','ar_EG_LNum','ar_ER','ar_EG','ar-EG','ca_ES','ca-ES','es_ES','et_EE','es-ES','en-EG','en-ES','en_ES','en_EE','et-EE','en-EN','es_EC','eu_ES','en_EG','en-EE','es-EC','en_EC','es_EA','fa_EG','gl_ES','ja-EH','ja-EC','ja-ES','ko-EE','ko_ES','ko-EG','ko-ES','ru_EE','ro-ES','ru-EE','th_EG','vi-EG','vi-ES','vi-EC','zh_ER_#Hans','en_FI','en-FR','en_FR','en_FJ','en-FJ','en-FI','fi-FI','fr_FR','fi_FI','fr-FR','fa_FA','fr_FH','it-FR','ja-FR','ko-FR','ko-FJ','ko-FI','pt-FR','sv_FI','tr-FR','th_FI','th_FR','th-FI','vi-FI','vi-FM','ar_GB','da_GL','en-GB','es_GT','en_GY','en_GR','en-GU','el-GR','en-GG','es-GB','en-GE','es-GT','en_GB','el_GR','es_GQ','en_GH','en_GU','en_GT','en-GR','en-GH','en_GI','fr_GA','fr_GP','fr_GB','fr_GF','fr-GB','ja-GB','ja-GI','ja-GU','ja-GF','ko-GH','ka-GE','ko-GB','ko-GG','ko-GU','ko-GA','ko-GP','ko-GL','ko-GT','ko-GD','ko-GN','ko_GB','ko-GW','ko-GR','ko-GM','ko-GY','ka_GE','lt-GB','pl-GB','ru_GB','ro-GB','ru-GB','ru-GR','ru_GE','ru-GE','tr_GB','th_GB','th-GR','th-GL','th_GL','th_GR','th-GB','uk-GB','vi-GE','vi-GB','vi-GP','ar-HK','en_HK','es_HN','en_HU','en-HK','en_HR','hu_HU','hr_HR','hu-HU','hr-HR','it-HK','ja-HK','ja-HN','ja-HU','ja_HK','ko-HK','ko-HU','ko_HU','ru-HK','th-HK','vi-HK','vi-HU','zh-Hans-CN','zh-Hans-US','zh-HK','zh-Hant-TW','zh-Hant-HK','zh-Hans-HK','zh-Hant-JP','zh-Hant-GB','zh-Hant-US','zh-Hans-GB','zh-Hans-KR','zh-Hant-TH','zh-Hans-TR','zh-Hans-FK','zh-Hans-RU','zh-Hans-NL','zh-Hans-IT','zh_HK_#Hans','zh-Hant-PG','zh-Hant-TR','zh-Hans-AC','zh-Hant-TT','zh-Hant-EC','zh-Hans-KE','zh-Hans-MM','zh-Hant-FR','zh-Hans-TH','zh-Hant-IT','zh-Hans-VN','zh-Hans-BR','zh-Hans-ET','zh-Hant-VN','zh-Hans-GP','zh-Hans-SE','zh-Hans-AF','zh-Hans-ES','zh-Hans-MX','zh-Hant-MY','zh-Hans-BE','zh-Hans-CF','zh-Hans-JE','zh-Hans-TG','zh-Hant-NA','zh-Hans-NA','zh-Hans-AO','zh-Hans-TT','zh-Hans-EE','zh-Hans-UA','zh-Hans-NF','zh-Hans-NO','zh-Hans-CP','zh-Hans-AD','zh-Hant-DK','zh-Hans-TL','zh-Hans-PG','zh-Hant-BR','zh-Hans-BS','zh-Hant-ER','zh-Hans-GY','zh-Hans-GI','zh-Hant-TM','zh-Hans-VE','zh-Hans-EC','zh-Hans-BQ','zh-Hans-GL','zh-Hant-NP','zh-Hans-IL','zh-Hant-TV','zh-Hans','zh_HK','zh-Hant','zh-Hant-CN','zh_HK_#Hant','zh-Hans-JP','zh-Hans-CA','zh-Hant-MO','zh-Hans-SG','zh-Hant-AU','zh-Hant-KR','zh-Hant-NZ','zh-Hant-ES','zh-Hans-MO','zh-Hans-MY','zh-Hans-MR','zh-Hans-TW','zh-Hans-AU','zh-Hans-AL','zh-Hant-CA','zh-Hans-DZ','zh-Hant-TC','zh-Hans-AR','zh-Hans-FR','zh-Hans-BN','zh-Hans-AS','zh-Hant-SG','zh-Hans-DE','zh-Hans-AW','zh-Hans-OM','zh-Hans-NZ','zh-Hant-NL','zh-Hans-TC','zh-Hans-MN','zh-Hans-AE','zh-Hans-TF','zh-Hant-CF','zh-Hans-KG','zh-Hans-AZ','zh-Hant-BT','zh-Hans-AI','zh-Hans-ID','zh-Hans-UM','zh-Hans-GA','zh-Hans-PH','zh-Hant-DO','zh-Hans-LA','zh-Hans-DK','zh-Hans-EG','zh-Hant-AS','zh-Hans-UG','zh-Hans-SA','zh-Hans-NG','zh-Hans-CZ','zh-Hans-BM','zh-Hans-IE','zh-Hant-ZA','zh-Hans-IS','zh-Hans-BT','zh-Hans-AT','zh-Hans-BI','zh-Hans-PA','zh-Hant-NU','zh-Hans-RE','zh-Hant-AL','zh-Hans-BO','zh-Hans-IN','zh-Hans-CM','zh-Hans-ZA','zh-Hans-MV','zh-Hans-TD','zh-Hans-RS','zh-Hans-CH','zh-Hant-GI','ar-IQ','ar_IQ_LNum','ar_IL','ar_IQ','bn_IN','en_IE','en-ID','en_IR','en-IE','es_IC','en-IT','en-IM','en-IL','es-IC','en_IS','en_IQ','en_IN','en_ID','en_IT','en-IN','en_IL','en-IQ','fa_IR','fa_IR_LNum','fr-IL','gu_IN','hi_IN','he-IL','it_IT','it-IT','is_IS','id-IC','ir_IR','in_ID','iw_IL','id-ID','iw_IS','jv_ID','ja-IT','ja-IE','ja-IS','ja-ID','ja-IL','ko-IT','ko_ID','ko-ID','ko_IN','ko_IQ','ko-IQ','ko-IN','ko-IE','ml_IN','mr_IN','or_IN','pa_IN_#Guru','pa_IN','ru_IT','ro-IT','ru-IL','sv-IQ','sv-IS','su_ID','ta_IN','te_IN','tr-IQ','ur_IN','vi-ID','vi-IN','vi-IE','zh_ID','zh_IT','ar_JO_LNum','ar-JO','ar_JO','de-JP','en-JP','en_JP','en_JM','en-JO','es-JP','en-JM','es_JP','en-JE','fr-JP','fr_JP','it-JP','id-JP','ja_JP','ja-JP','ja_JA','ja-JO','ko_JP','ko-JP','pt-JP','ru-JP','th-JP','th_JP','vi-JP','vi_JP','zh_JP','ar_KW','ar-KW','de-KR','en-KH','en_KH','en_KW','en-KR','en-KW','en_KR','es-KR','en_KE','en_KY','en-KG','fr-KR','hi-KR','io-KR','id-KR','it-KR','ja-KG','ja_KR','ja-KR','ja-KH','ko_KR','km-KH','ko-KG','ko-KE','ko_KW','ko-KZ','ko-KR','km_KH','ko_KP','ko-KW','ko-KH','kk_KZ','ko_KO','ko_KH','mn-KR','ru_KZ','ru-KR','ru-KG','ru_KG','ru-KZ','th_KR','tr-KR','th-KR','th_KH','th-KH','vi-KH','vi_KR','vi-KR','vi_KH','vi-KI','zh_KR','zh_KH','ar_LB','ar_LY','ar-LY','de-LU','de_LI','de_LU','en_LT','en_LK','en-LT','en-LY','en-LA','en_LA','en_LV','en-LB','en_LU','en_LB','fr_LU','ha_LA','ja-LU','ja-LT','ja-LK','ko-LA','ko-LI','ko_LA','ko-LK','lt_LT','lv-LV','lo_LA','lv_LV','ru_LV','si_LK','th-LA','th_LA','vi-LA','ar_MA','ar_MR','ar-MY','bo-MO','es_MX','en_MY','en-MO','en-MN','en_MU','en-MM','en-MA','en_MM','en-MX','en_MA','es-MX','en-MY','en_MX','en_MO','en-MT','en_MT','en_MW','en_MV','en_MH','en-MK','fr_MA','fr_MU','fr_MQ','fr_MC','fr-MQ','fr-MA','in_MY','ja-MY','ja-MC','ja-MA','ja-MO','ja-MX','ko-MN','ko-MM','ko-MV','ko_MY','ko-MP','ko_MX','ko-MO','ko-MC','ko-MT','ko-MY','ko-MX','ms_MY','mk_MK','mn_MN','my_MM','pt_MZ','ru-MD','ru_MD','ro_MD','sr_ME_#Latn','tr-MT','vi-MY','vi-MU','vi-MO','zh_MO_#Hant','zh_MO_#Hans','zh_MM','zg_MM','zh_MX','zh_MY_#Hans','zh_MO','zh_MY','zh_MY_#Hant','ar-NE','de-NO','en_NZ','en-NZ','es_NI','en-NO','en-NP','en_NA','en-NI','en-NA','en_NL','en-NL','en_NP','en-NG','en_NO','en_NG','fr_NC','ja-NA','ja-NL','ja-NF','ja-NZ','ko-NC','ko-NL','ko-NP','ko-NG','ko-NO','ko-NZ','ko-NI','ko_NO','ko_NL','ko-NA','ko_NZ','nb_NO','nl_NL','nl-NL','nb-NO','pt-NL','th_NO','vi-NL','vi-NO','vi-NZ','ar_OM','en-OM','en_OM','ar-PS','ar_PS','en_PA','en-PH','en_PT','en_PR','en_PK','es-PY','en_PL','en-PT','es-PA','en-PA','en_PH','es_PE','es_PA','es_PY','en-PK','en_PE','es_PR','en-PE','en_PG','es-PR','es-PE','en-PR','en-PL','fr-PF','in_PH','ja-PH','ko_PH','ko-PL','ko-PE','ko_PY','ko-PH','ko_PL','pl_PL','pt_PT','pl-PL','pt-PH','pt-PT','ru-PH','sv-PL','th_PL','tl_PH','th-PN','ur_PK','vi-PH','vi_PH','vi-PL','zh_PH','ar-QA','ar_QA','en_QA','en-QA','fr-QA','ko-QA','th_QA','en_RU','en-RS','en-RO','en_RO','en-RU','en_RS','fr_RE','fr-RU','hr-RS','ja-RU','ko_RU','ko-RU','ru_RU','ro_RO','ro-RO','ru-RU','sr_RS','sr_RS_#Latn','sr_RS_#Cyrl','tr-RU','vi-RU','vi-RE','ar_SA','ar_SA_LNum','ar-SA','ar_SY','ar_SD','en_SG','en_SK','en-SK','en_SD','en-SA','en_SY','en_SE','es_SV','en-SG','en-SE','en_SI','en_SA','fr_SN','fr-SE','it_SM','ja_SG','ja-SG','ja-SN','ja-SS','ko-SA','ko-SS','ko_SG','ko-SB','ko-SG','ko-SK','nl_SR','nb_SJ','pl_SP','sl-SI','sk_SK','sv_SE','sl_SI','sv-SE','sk-SK','th-SG','th_SG','th_SD','vi-SG','vi-SA','vi_SH','vi-SE','vi-SL','zh_SG_#Hans','zh_SG','ar_TD','ar_TN','en-TW','en_TW','en_TT','en_TZ','en_TH','en-TH','en-TZ','en-TR','en_TR','en_TN','en-TT','en_TJ','fr_TN','fa_TH','fr_TG','ja-TH','ja_TW','ja_TH','ja-TW','ja-TC','ko-TW','ko-TH','ko_TW','km-TH','ko-TR','ko_TH','ko-TF','ru_TJ','ru-TW','th_TH','tr_TR','tr-TR','th-TW','th-TR','th-TH','tr-TW','vi-TH','vi-TW','vi_TW','vi-TV','zh-TW','zh_TH','zh_TW','zh_TW_#Hant','ar_US','ar-US','de-US','de_US','es_US','es_UY','en_UA','en_UY','en_US','en-US','es-US','en_UK','en_UM','es-UY','en-UM','en-UA','en_UG','en-UZ','fr-US','hd_US','in_US','id-US','it-US','ja-UZ','ja-US','ja-UM','ja_US','ko_US','ko-US','ko-UY','ko-UM','ko_UK','pt-US','pt_US','ru_US','ru_UA','ru-UA','ru-UZ','ru-US','ru_UZ','ru_US_us','th_UK','th-US','tr_US','th_US','tr-US','to-US','uk_UA','uk-US','uk_UK','uk-UA','uz_UZ','uz_UZ_#Latn','vi-US','vi_US','vn_US','vi-UA','vi-UM','xh_US','zh_US','zh_UK','de-VN','en_VN','en_VI','en_VE','es-VE','en_VG','en_VU','en-VN','es_VE','en-VE','en-VG','fr-VN','in_VN','ja-VN','ja_VN','ja-VA','ko-VN','ko-VI','ko-VE','ko_VN','pl-VN','th_VN','vi-VE','vn_VN','vi_VN','vi-VN','vi_VI','vi-VG','vi-VI','zh_VN','vi-WF','ar_XB','en_XA','sq_XK','sq-XK','ar_YE','ar-YE','fr_YT','ar_ZW','af_ZA','af-ZA','en_ZG','en-ZW','en_ZW','en_ZA','en-ZA','en_ZM','ko-ZA','my_ZG','pt-ZA','vi-ZW','vi-ZM','zz_ZZ','en__US','fil_PH','und_CN','yue_HK','libar','ppedf','None','unity'}
#valid_codes = {'iPhone10,4','Samsung SM-G955N','Asus ASUS_Z011D','Samsung SM-N950N','Samsung SM-T820','iPhone10,2','iPhone10,3','iPhone10,6'}

# test
with tf.Session() as sess:
  saver.restore(sess, checkpoint_dir)
  test_util.test('test', sess, test_data[:1000])

INFO:tensorflow:Restoring parameters from ./training_checkpoints
[('iPad6,4', 43.970474), ('iPhone7,2', 49.95916), ('iPad4,5', 50.919895), ('iPad4,4', 52.645172), ('iPad5,3', 54.3596), ('Samsung SM-N910G', 55.879066), ('Samsung SM-G910S', 56.691895), ('Samsung SM-G950W', 57.638115), ('Samsung SM-N920V', 57.703358), ('LG Gx2', 58.753056), ('iPhone10,5', 59.042206), ('iPad6,3', 59.29645), ('Samsung SM-N9100', 59.74781), ('Samsung SM-G900V', 62.10347), ('BQ-5058', 62.91767), ('Samsung SM-G925S', 63.2264), ('Samsung SM-T820', 63.883186), ('HTC A2w', 63.950043), ('Vernee thor', 64.18054), ('OPPO D4', 64.223), ('Samsung SM-G920L', 64.24934), ('Samsung SM-N935S', 64.78363), ('Samsung SM-T710', 65.452614), ('Sony E5563', 66.36438), (' 13Plus', 67.47608), ('Vivo Y31', 67.673706), ('Samsung SM-G928T', 67.93773), ('Samsung SM-G710S', 68.13241), ('Samsung SM-N7507', 68.5354), ('Vivo Y75A', 68.77115), ('xiaomi hm 2a', 68.877174), ('Oppo r7s', 69.42723), ('Samsung SM-G928F', 70.23003), ('iPad4,7', 7

In [16]:
# generation
gen_count = 10000
gen_codes = vae.make_decoder(prior.sample(gen_count)).mean()

with tf.Session() as sess:
  saver.restore(sess, checkpoint_dir)
  #test_util.test('generate', sess, samples)
  result = {}
  for i in range(1) :
    samples = sess.run(gen_codes)
    ascii_codes_list = test_util.convert_onehot_to_ascii(sess, samples)
    print(ascii_codes_list)
    score = test_util.get_score_list(sess, samples)
    for j, ascii_code in enumerate(ascii_codes_list) :
      result[ascii_code] = score[j]
  sortable = []
  for key in result:
    value = result[key]
    sortable.append((key,value))
  sorted_by_value = sorted(sortable, key=lambda x: x[1])
  print(sorted_by_value)

d_input Tensor("8ceb2632-7d68-4cdc-8fa0-ab117c34ed6e_1/Reshape:0", shape=(10000, 20, 32, 32), dtype=float32)
d_output Tensor("8ceb2632-7d68-4cdc-8fa0-ab117c34ed6e_1/conv2d_transpose/BiasAdd:0", shape=(10000, 20, 64, 16), dtype=float32)
d_output Tensor("8ceb2632-7d68-4cdc-8fa0-ab117c34ed6e_1/conv2d_transpose_1/BiasAdd:0", shape=(10000, 20, 128, 1), dtype=float32)
d_output Tensor("8ceb2632-7d68-4cdc-8fa0-ab117c34ed6e_1/Reshape_1:0", shape=(10000, 20, 128), dtype=float32)
INFO:tensorflow:Restoring parameters from ./training_checkpoints
['SEa unE SM-G9~W~~E~~', 'SEa unE SM-~9~W~~E~~', 'SEa 1nE SM-G9,W~SE~~', 'SEa unE SM-~9~W~~E~~', '\x01\x01\x01\x01\x01\x011,SM-.~,~~~~~~', 'S\x01\x01\x01\x01\x01E,SM-.9,~~~M~~', 'SEa unE SM-G9,W~SE~~', '\x01\x01\x01\x01\x01\x01\x01,.M,.,,~~~~~~', '\x01\x01\x01\x01\x01\x01\x01,SM-.~,~~~~~~', 'SEa unE SM-~9~W~~E~~', 'SEm 1nE,SM-.9,W1SM~~', 'SEa 1nE SM-.9,W~SM~~', '\x01\x01\x01\x01\x01\x01\x01,.2..~~~~~~~~', 'SE\x01 \x01nE SM-~9~~~~M~~', '\x01\x01\x01\x01\x01\

In [17]:
# generation from manually created codes
manual_codes = tf.placeholder(tf.float32, [None, code_size])
gen_codes_manual = vae.make_decoder(manual_codes).mean()

with tf.Session() as sess:
  saver.restore(sess, checkpoint_dir)
  codes=[\
        [-0.24404454,1.7446337],
        [0,0],
         [-1,-1]
  ]
  #codes = sess.run(prior.sample(10))
  samples = sess.run(gen_codes_manual, {manual_codes:codes})
  test_util.test('generate', sess, samples)

d_input Tensor("8ceb2632-7d68-4cdc-8fa0-ab117c34ed6e_2/Reshape:0", shape=(?, 20, 32, 32), dtype=float32)
d_output Tensor("8ceb2632-7d68-4cdc-8fa0-ab117c34ed6e_2/conv2d_transpose/BiasAdd:0", shape=(?, 20, 64, 16), dtype=float32)
d_output Tensor("8ceb2632-7d68-4cdc-8fa0-ab117c34ed6e_2/conv2d_transpose_1/BiasAdd:0", shape=(?, 20, 128, 1), dtype=float32)
d_output Tensor("8ceb2632-7d68-4cdc-8fa0-ab117c34ed6e_2/Reshape_1:0", shape=(?, 20, 128), dtype=float32)
INFO:tensorflow:Restoring parameters from ./training_checkpoints
[('WIa 0/I0WQ-/=/0//I/~', 76.43107), ('\x01\x01\x01\x01\x01\x01\x0102Q-0000~~~~~', 121.30853), ('W\x01\x01\x010\x01I0WQ-*=/0~~Q~~', 137.68942)]


In [20]:
# generation from manually created codes
manual_codes = tf.placeholder(tf.float32, [None, code_size])
gen_codes_manual = vae.make_decoder(manual_codes).mean()

with tf.Session() as sess:
  saver.restore(sess, checkpoint_dir)
  codes=[\
         [-1.5,-1.5], [-1.5,-1],[-1.5,-0.5],[-1.5,0],[-1.5,0.5],[-1.5,1],
         [-1,-1.5],   [-1,-1],  [-1,-0.5],[-1,0],[-1,0.5],[-1,1],
         [-0.5,-1.5], [-0.5,-1],[-0.5,-0.5],[-0.5,0],[-0.5,0.5],[-0.5,1],
         [0,-1.5],    [0,-1],   [0,-0.5],[0,0],[0,0.5],[0,1],
         [0.5,-1.5],  [0.5,-1], [0.5,-0.5],[0.5,0],[0.5,0.5],[0.5,1],
         [1,-1.5],    [1,-1],   [1,-0.5],[1,0],[1,0.5],[1,1],
         [1.5,-1.5],  [1.5,-1], [1.5,-0.5],[1.5,0],[1.5,0.5],[1.5,1],
         [2,-1.5],  [2,-1], [2,-0.5],[2,0],[2,0.5],[2,1],
  ]
  #codes = sess.run(prior.sample(10))
  samples = sess.run(gen_codes_manual, {manual_codes:codes})
  ascii_codes_list = test_util.convert_onehot_to_ascii(sess, samples)
  line = ''
  count = 0
  width = 6
  for v in ascii_codes_list :
    line = line + '\t' + v
    count = count + 1
    if count == width :
      print(line)
      count = 0
      line = ''

d_input Tensor("8ceb2632-7d68-4cdc-8fa0-ab117c34ed6e_4/Reshape:0", shape=(?, 20, 32, 32), dtype=float32)
d_output Tensor("8ceb2632-7d68-4cdc-8fa0-ab117c34ed6e_4/conv2d_transpose/BiasAdd:0", shape=(?, 20, 64, 16), dtype=float32)
d_output Tensor("8ceb2632-7d68-4cdc-8fa0-ab117c34ed6e_4/conv2d_transpose_1/BiasAdd:0", shape=(?, 20, 128, 1), dtype=float32)
d_output Tensor("8ceb2632-7d68-4cdc-8fa0-ab117c34ed6e_4/Reshape_1:0", shape=(?, 20, 128), dtype=float32)
INFO:tensorflow:Restoring parameters from ./training_checkpoints
	OIKKOKKKOMKJOKOOKOHL	HIKKOKIKOMKJKKKKKMHL	HImKuKIKLMKJHKKKKM0L	SPmsuIg LM-J9K0KKE01	SPmsueg SM-J9K0LEE01	SPmsung SM-G950N2E21
	OIKKOKKKOMKJOKOOKOHL	HIKKOKIKOMKJOKKKKMHL	HImKuKIKLMKJHKKKKM0L	SPmsueg LM-J9K0KKE01	SPmsueg SM-G950L2E01	SPmsung SM-G950N2E21
	OIKKOKKKOMKJOKOOKOHL	OIKKOKIKOMKJOKKKKMHL	HImKuIIKLMKJHKKKKM0L	SPmsueg SM-J9K0LEE04	SPmsueg SM-G950N2E21	SPmsung SM-G950N1E21
	OIKsOIIKOMKJOKKOKOOL	SIKKOIIKOMKJOKKKKMHL	SamsuII LMHJHKKKKM0L	Samsueg SM-J9K0LEE04	SPmsueg SM-

In [10]:
# generation from char sequences
charseq = ['iPad6,4', 'iPhone','Samsung']
onehot = preprocess.convert_str_array_to_onehot_ndarray('gen', charseq)
manual_codes = tf.placeholder(tf.float32, [None, code_size])
gen_codes_manual = vae.make_decoder(manual_codes).mean()
with tf.Session() as sess:
  saver.restore(sess, checkpoint_dir)
  codes = sess.run([code], {data:onehot})[0]
  samples = sess.run(gen_codes_manual, {manual_codes:codes})
  test_util.test('generate', sess, samples)

d_input Tensor("4457e5f7-908b-458e-828a-0d71b6f84478_4/Reshape:0", shape=(?, 20, 32, 32), dtype=float32)
d_output Tensor("4457e5f7-908b-458e-828a-0d71b6f84478_4/conv2d_transpose/BiasAdd:0", shape=(?, 20, 64, 16), dtype=float32)
d_output Tensor("4457e5f7-908b-458e-828a-0d71b6f84478_4/conv2d_transpose_1/BiasAdd:0", shape=(?, 20, 128, 1), dtype=float32)
d_output Tensor("4457e5f7-908b-458e-828a-0d71b6f84478_4/Reshape_1:0", shape=(?, 20, 128), dtype=float32)
INFO:tensorflow:Restoring parameters from ./training_checkpoints
[('Samsung SM-G950NNNN1', 41.56355), ('Samsung SM-G950NNmum', 62.485905), ('Samsung SM-G950NNmum', 63.82652)]
